In [1]:
import pandas as pd
import sqlalchemy 
import psycopg2
import os
import numpy as np
import requests
import re
import copy
from pandas.api.types import is_numeric_dtype
from sklearn.linear_model import LinearRegression
from sqlalchemy import create_engine
import datetime
from datetime import datetime as dt
import sys
import importlib

In [2]:
sys.path.append('..')

In [3]:
import modules.transforms

In [4]:
importlib.reload(modules.transforms)

<module 'modules.transforms' from '/home/jovyan/work/ETL/../modules/transforms.py'>

In [5]:
from modules.transforms import *

In [6]:
#guardar paso en csv
path='/data/ETLcache/RAW'
now = dt.now()
timestamp = now.strftime("_%d%m%Y_%H%M%S")

In [7]:
#OBTENER RUTAS DE CARPETAS DISPONIBLES EN REPO COVID DE MINISTERIO DE CIENCIAS DE CHILE
url="https://github.com/MinCiencia/Datos-COVID19/tree/master/output/"
request=requests.get(url)
root=url.split('github.com')[1]
prefix=url.split('/Min')[0]
folders=request.text.split(root)

#product folders
for idx in range(len(folders)):
    f=folders[idx]
    f=f.split('">')[0]
    folders[idx]=f
folders=[prefix+root+f for f in folders if 'producto' in f]

In [8]:
#DEFINIR LISTADO DE PRODUCTOS QUE SE VAN A CONSULTAR DE REPO DE MINISTERIO DE CIENCIAS

productos=['producto5',
           #'producto3',
           #'producto10',
           #'producto13',
           #'producto25',
           'producto14',
           'producto19',
           'producto37',
           'producto53',
           'producto54',
           'producto52',
           'producto50',
           'producto48',
           'producto24'
    
]

In [9]:
#LISTADO DE CARPETAS (RUTAS) FILTRADO, SEGÚN PRODUCTOS A CONSULTAR
fs=[]
for p in productos:
    fs.extend([ x for x in folders if p==x.rsplit('/',1)[1]])

In [13]:
#LEER ARCHIVOS DENTRO DE RUTAS CORRESPONDIENTES A PRODUCTOS, GUARDARLOS EN DICCIONARIO "TABLES"
tables={}
for f in fs:
    files_folder=f.rsplit('/')[-1]
    files_folder_prefix='/MinCiencia/Datos-COVID19/blob/master/output/'+files_folder+'/'
    tables[files_folder]={}
    raw_path='https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/'+files_folder+'/'
    req=requests.get(f)
    files=req.text.split(files_folder_prefix)
    
    for idx in range(len(files)):
        files[idx]=files[idx].split('">')[0]
    files=[raw_path+f for f in files if '.csv' in f]
    for fl in files:
        try:
            print('reading file: '+fl)
            table=pd.read_csv(fl)
            table_name='_'.join(fl.rsplit('/',2)[-2:]).strip('.csv')
            

            
            outpath=path+'/'+files_folder
            
            if not os.path.exists(outpath):
                os.makedirs(outpath)
            
            
            outfile=outpath+'/'+table_name+timestamp+'.csv'
            print("saving raw table"+outfile+'in cache')
            table.to_csv(outfile)
            
        except Exception as e:
            print(str(e))
            pass

reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto5/TotalesNacionales.csv
saving raw table/data/ETLcache/RAW/producto5/producto5_TotalesNacionale_17022022_205942.csvin cache
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto5/TotalesNacionales_T.csv
saving raw table/data/ETLcache/RAW/producto5/producto5_TotalesNacionales_T_17022022_205942.csvin cache
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto5/TotalesNacionales_std.csv
saving raw table/data/ETLcache/RAW/producto5/producto5_TotalesNacionales_std_17022022_205942.csvin cache
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto14/FallecidosCumulativo.csv
saving raw table/data/ETLcache/RAW/producto14/producto14_FallecidosCumulativo_17022022_205942.csvin cache
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto14/Fa

saving raw table/data/ETLcache/RAW/producto50/producto50_DefuncionesDEIS_confirmadosPorComuna_std_17022022_205942.csvin cache
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto50/DefuncionesDEIS_sospechososPorComuna.csv
saving raw table/data/ETLcache/RAW/producto50/producto50_DefuncionesDEIS_sospechososPorComuna_17022022_205942.csvin cache
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto50/DefuncionesDEIS_sospechososPorComuna_T.csv
saving raw table/data/ETLcache/RAW/producto50/producto50_DefuncionesDEIS_sospechososPorComuna_T_17022022_205942.csvin cache
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto50/DefuncionesDEIS_sospechososPorComuna_std.csv
saving raw table/data/ETLcache/RAW/producto50/producto50_DefuncionesDEIS_sospechososPorComuna_std_17022022_205942.csvin cache
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master

In [14]:
os.system('jupyter nbconvert --output /home/jovyan/work/ETLdocs/' + 'ETL_covid-chile.html' + ' --to html ' + 
          '/home/jovyan/work/ETL/1-RAW.ipynb')

[NbConvertApp] Converting notebook /home/jovyan/work/ETL/1-RAW.ipynb to html
[NbConvertApp] Writing 605061 bytes to /home/jovyan/work/ETLdocs/ETL_covid-chile.html


0